In [15]:
import numpy as np
import skimage
import utils
import pathlib
import matplotlib.pyplot as plt
import math

In [227]:
def otsu_thresholding(im: np.ndarray) -> int:
    """
        Otsu's thresholding algorithm that segments an image into 1 or 0 (True or False)
        The function takes in a grayscale image and outputs a threshold value

        args:
            im: np.ndarray of shape (H, W) in the range [0, 255] (dtype=np.uint8)
        return:
            (int) the computed thresholding value
    """
    assert im.dtype == np.uint8
    ### START YOUR CODE HERE ### (You can change anything inside this block) 
    # You can also define other helper functions
    #Step one: Normalized Histogram
    def norm_hist(im):
        low = 0
        high = 255
        bins = np.linspace(low, high, high - low + 1)
        hist, bin_edges = np.histogram(im,bins=bins, density=True)
        hist_norm = hist*np.diff(bin_edges)
        return hist_norm
    #Step Two: Cumulative Sum
    def cum_sum(hist,threshold):
        p1 = hist[0:threshold].sum()
        p2 = hist[threshold:len(hist)].sum()
        return p1,p2
    #Step Three: Cumulative Means
    def cum_mean(hist,threshold):
        m1 = np.sum(hist[0:threshold]*np.arange(0,threshold,1))
        m2 = np.sum(hist[threshold:len(hist)]*np.arange(threshold,len(hist),1))
        return m1, m2
    #Step Four: Global Means
    def global_mean(hist):
        return (hist*np.arange(0,len(hist),1)).sum()
    #Step Five: Between class variance
    def between_class_var(mg, m1, p1):
        variance = ((mg*p1)-m1)**2/(p1*(1-p1))
        return variance

    # Compute normalized histogram
    #Normalized Histogram
    hist = norm_hist(im)
    #Global Means
    mg = global_mean(hist)
    #Dictionary to store Threshold value and respective Between class variance
    var_dict = {}
    #Iterating over threshold values
    for x in range(1, len(hist)):
        p1, p2 = cum_sum(hist, x)
        #If p1 is valid, other wise we will get invalid values for zero division
        if 0<p1<1:
            #Cumulative mean
            m1, m2 = cum_mean(hist, x)
            #Between class variance
            var = between_class_var(mg, m1, p1)
            #Storing variance value
            var_dict[x] = var
    #Sorting dictionary from maximum to minimum values
    var_list_sorted = sorted(var_dict.items(), key=lambda x: x[1], reverse=True)
    #Listing thresholds of max variance values
    max_list = [k for k,v in var_list_sorted if v == var_list_sorted[0][1]]
    #Average of max threshold values
    optimal_threshold = sum(max_list)/len(max_list)

    return optimal_threshold
    ### END YOUR CODE HERE ###

In [228]:
if __name__ == "__main__":
    # DO NOT CHANGE
    impaths_to_segment = [
        pathlib.Path("thumbprint.png"),
        pathlib.Path("polymercell.png")
    ]
    for impath in impaths_to_segment:
        im = utils.read_image(impath)
        threshold = otsu_thresholding(im)
        print("Found optimal threshold:", threshold)

        # Segment the image by threshold
        segmented_image = (im >= threshold)
        assert im.shape == segmented_image.shape, "Expected image shape ({}) to be same as thresholded image shape ({})".format(
                im.shape, segmented_image.shape)
        assert segmented_image.dtype == bool, "Expected thresholded image dtype to be np.bool. Was: {}".format(
                segmented_image.dtype)

        segmented_image = utils.to_uint8(segmented_image)

        save_path = "{}-segmented.png".format(impath.stem)
        utils.save_im(save_path, segmented_image)

Reading image: images/thumbprint.png
Found optimal threshold: 154.0
Saving image to: image_processed/thumbprint-segmented.png
Reading image: images/polymercell.png
Found optimal threshold: 182.0
Saving image to: image_processed/polymercell-segmented.png
